In [1]:
from s2h_utils import *
import time

## Load data

In [2]:
data = pd.read_excel("C:/Users/sawal/Downloads/data_at.xlsx")[["id", "DT_DEB_PER_INA","DT_FIN_PER_INA"]]
data.columns = ["id", "date_debut", "date_fin"]
data['date_debut'] = pd.to_datetime(data['date_debut'])
data['date_fin'] = pd.to_datetime(data['date_fin'])
data

,id,date_debut,date_fin
0,1.796940e+19,2022-02-15,2022-02-19
1,9.984544e+18,2021-12-15,2021-12-19
2,9.984544e+18,2021-12-15,2021-12-18
3,9.984544e+18,2022-01-21,2022-01-30
4,9.984544e+18,2022-01-21,2022-01-30
...,...,...,...
3361,9.310159e+18,2021-01-27,2021-02-07
3362,9.310159e+18,2022-03-22,2022-03-25
3363,9.310159e+18,2022-03-22,2022-03-25
3364,1.385217e+19,2018-10-03,2018-10-08


In [3]:
# creer les données calendaires par individus : du premier mois d'abscence ou dernier mois
baseline = data.groupby("id").agg({'date_debut':'min', 'date_fin':'max'}).reset_index()
baseline['date_debut'] = baseline['date_debut'].agg(['min'])[0] - pd.offsets.MonthBegin()
baseline['date_fin'] = baseline['date_fin'].agg(['max'])[0] + pd.offsets.MonthEnd()

In [4]:
baseline

,id,date_debut,date_fin
0,6.541784e+15,2013-09-01,2022-12-31
1,6.917830e+16,2013-09-01,2022-12-31
2,1.129997e+17,2013-09-01,2022-12-31
3,1.243011e+17,2013-09-01,2022-12-31
4,1.385116e+17,2013-09-01,2022-12-31
...,...,...,...
391,1.830080e+19,2013-09-01,2022-12-31
392,1.831310e+19,2013-09-01,2022-12-31
393,1.833341e+19,2013-09-01,2022-12-31
394,1.835622e+19,2013-09-01,2022-12-31


In [5]:
#generer les dates journalieres associées à chaque ligne d'abscence.
data['New_Date'] = data.apply(lambda x: pd.date_range(x.date_debut, x.date_fin, freq='1d'), axis=1)
# explode : from wide to long data
data = data.explode('New_Date')
data

,id,date_debut,date_fin,New_Date
0,1.796940e+19,2022-02-15,2022-02-19,2022-02-15
0,1.796940e+19,2022-02-15,2022-02-19,2022-02-16
0,1.796940e+19,2022-02-15,2022-02-19,2022-02-17
0,1.796940e+19,2022-02-15,2022-02-19,2022-02-18
0,1.796940e+19,2022-02-15,2022-02-19,2022-02-19
...,...,...,...,...
3364,1.385217e+19,2018-10-03,2018-10-08,2018-10-06
3364,1.385217e+19,2018-10-03,2018-10-08,2018-10-07
3364,1.385217e+19,2018-10-03,2018-10-08,2018-10-08
3365,1.070249e+18,2022-02-15,2022-02-16,2022-02-15


In [6]:
# Recuperer les jours de congés par id sans doublons
data[["id","New_Date"]].drop_duplicates()

,id,New_Date
0,1.796940e+19,2022-02-15
0,1.796940e+19,2022-02-16
0,1.796940e+19,2022-02-17
0,1.796940e+19,2022-02-18
0,1.796940e+19,2022-02-19
...,...,...
3364,1.385217e+19,2018-10-06
3364,1.385217e+19,2018-10-07
3364,1.385217e+19,2018-10-08
3365,1.070249e+18,2022-02-15


**Donc la somme des NB_arret doit etre égale 46485 . C'est la réference pour tester si notre solution marche ou pas**